### 增加数据

降低模型方差的最直接方法就是增加训练数据(但注意验证集和测试集合与之前保持一致)

In [1]:
import numpy as np


class DataBinWrapper(object):
    def __init__(self, max_bins=10):
        # 分段数
        self.max_bins = max_bins
        # 记录x各个特征的分段区间
        self.XrangeMap = None

    def fit(self, x):
        n_sample, n_feature = x.shape
        # 构建分段数据
        self.XrangeMap = [[] for _ in range(0, n_feature)]
        for index in range(0, n_feature):
            tmp = sorted(x[:, index])
            for percent in range(1, self.max_bins):
                percent_value = np.percentile(tmp, (1.0 * percent / self.max_bins) * 100.0 // 1)
                self.XrangeMap[index].append(percent_value)
            self.XrangeMap[index] = sorted(list(set(self.XrangeMap[index])))

    def transform(self, x):
        """
        抽取x_bin_index
        :param x:
        :return:
        """
        if x.ndim == 1:
            return np.asarray([np.digitize(x[i], self.XrangeMap[i]) for i in range(0, x.size)])
        else:
            return np.asarray([np.digitize(x[:, i], self.XrangeMap[i]) for i in range(0, x.shape[1])]).T

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, auc
from pre_process.pre_process_wy1_beforejt3000_joincx import *
import toad
from sklearn import metrics
diretory="xxx"
test0 = pd.read_csv(diretory+'xxx1.csv', low_memory=False)
#追加部分新数据
other_df = pd.read_csv(diretory+'xxx2.csv', low_memory=False)
other_df=other_df[other_df['effdate'].apply(lambda x:x[:7]<="2019-10")]
test0=pd.concat([test0,other_df])

test0 = test0[(test0['dpt'] == 217) & (test0['renew_flag'] == 1)]

claim = pd.read_csv(diretory+'/data/claimnum.csv')
claim = claim[['polnum_com', 'num']]
claim = claim.rename(columns={'polnum_com': 'policy_no'})
test0=pd.merge(test0,claim,on="policy_no",how="inner")
test0['y'] = np.where(test0['num']>0, 1, 0)
test0=test0.drop("num",axis=1)
dat1=test0.copy()
#因子筛选
dat1 = ppcess_filterCols(dat1, cols='join')
#去掉饱和度过低的因子 by 80%
na_cols = []
for c in dat1.columns:
    if dat1[c].isnull().sum()/len(dat1)>0.8:
        na_cols.append(c)
dat1 = dat1.drop(na_cols, axis=1)
jt_cx_cols = ['xxx']
dat1 = dat1.drop(jt_cx_cols, axis=1)
dat1 = ppcess_dateCols(dat1,print_process_cols=False) # 日期因子格式
dat1 = ppcess_toNum(dat1,print_process_cols=False) # 部分因子转换为数值型
dat1 = ppcess_delCols_na(dat1,print_process_cols=False) # 删去饱和度低于1%的因子
dat1 = ppcess_inpute(dat1,print_process_cols=False) # 部分因子按规则进行空值填充，数值型因子在后面做均值填充
dat1 = ppcess_delCols_uniValue(dat1,print_process_cols=False) # 删去全部唯一值的因子

dat2 = dat1.copy()
dat2 = feature_catValues(dat2,print_process_cols=False) # 修正部分因子取值
dat2 = feature_factorLabel1(dat2,print_process_cols=False) # 离散特征编码1
dat2 = feature_factorLabel2(dat2,print_process_cols=False) # 离散特征编码2
dat2 = feature_lambda(dat2,print_process_cols=False) # 构建一些特征
dat2 = dat2.drop(['xxx'], axis=1)
dat2['xxx'] = dat2['xxx'].fillna(0)
dat2['y']=test0['y']
#切分训练、验证、测试
trn_df=dat2[test0['effdate'].apply(lambda x:x[:7]<="2019-10")]

dev_test_df=dat2[test0['effdate'].apply(lambda x:x[:7]>="2019-11")]
indice=list(range(0,dev_test_df.shape[0]))
np.random.shuffle(indice)
dev_test_df=dev_test_df.iloc[indice]
dev_df=dev_test_df.iloc[:dev_test_df.shape[0]//2]
test_df=dev_test_df.iloc[dev_test_df.shape[0]//2:]

#target encoding
object_cols=trn_df.dtypes[trn_df.dtypes==object].reset_index()['index'].tolist()
trn_df[object_cols]=trn_df[object_cols].fillna("missing")
dev_df[object_cols]=dev_df[object_cols].fillna("missing")
test_df[object_cols]=test_df[object_cols].fillna("missing")
object_target_cols=[]
for col in object_cols:
    object_target_cols.append(col+"_target")
    trn_df[col+"_target"]=trn_df[col]
    dev_df[col+"_target"]=dev_df[col]
    test_df[col+"_target"]=test_df[col]
import category_encoders as ce
le=ce.TargetEncoder(cols=object_target_cols)
le.fit(trn_df,trn_df['y'])
trn_df=le.transform(trn_df)
dev_df=le.transform(dev_df)
test_df=le.transform(test_df)
#ordinary encoding
oe=ce.OrdinalEncoder()
oe.fit(trn_df,cols=object_cols)
trn_df=oe.transform(trn_df)
dev_df=oe.transform(dev_df)
test_df=oe.transform(test_df)
#分箱做一次WOE
trn_woe_df=trn_df.drop(["policy_no","y"],axis=1).copy()
dev_woe_df=dev_df.drop(["policy_no","y"],axis=1).copy()
test_woe_df=test_df.drop(["policy_no","y"],axis=1).copy()
woe_cols=[item+"_woe" for item in trn_woe_df.columns]
dbw=DataBinWrapper()
dbw.fit(trn_woe_df.values)
trn_woe_df=pd.DataFrame(data=dbw.transform(trn_woe_df.values),columns=woe_cols)
dev_woe_df=pd.DataFrame(data=dbw.transform(dev_woe_df.values),columns=woe_cols)
test_woe_df=pd.DataFrame(data=dbw.transform(test_woe_df.values),columns=woe_cols)
trn_woe_df=trn_woe_df.astype("object")
dev_woe_df=dev_woe_df.astype("object")
test_woe_df=test_woe_df.astype("object")
woe_encoder=ce.WOEEncoder()
woe_encoder.fit(trn_woe_df,trn_df['y'],cols=woe_cols)
trn_woe_df=woe_encoder.transform(trn_woe_df)
dev_woe_df=woe_encoder.transform(dev_woe_df)
test_woe_df=woe_encoder.transform(test_woe_df)
trn_df=pd.concat([trn_df.reset_index(),trn_woe_df.reset_index()],axis=1).drop(["index"],axis=1)
dev_df=pd.concat([dev_df.reset_index(),dev_woe_df.reset_index()],axis=1).drop(["index"],axis=1)
test_df=pd.concat([test_df.reset_index(),test_woe_df.reset_index()],axis=1).drop(["index"],axis=1)

训练模型

In [3]:
#自定义metrics
def eval_function(y_true,y_pred):
    try:
        y_pred=y_pred.get_label()
    except:
        pass
    sort_indice=np.argsort(y_pred)[::-1]
    metric_value=y_true[sort_indice[:int(0.05*len(y_true))]].mean()
    return "eval_function",metric_value,True
def eval(y_true,y_pred):
    return np.round(eval_function(y_true,y_pred)[1]/eval_function(y_true,y_true)[1],2)

In [1]:
# 调参推荐：https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
# 超参
params = {
'boosting_type':'gbdt',#集成方式，还包括rf,dart,goss等
'objective':'binary',
'metric':"eval_function",
'learning_rate':0.01,#学习率
'max_depth':9,#单颗树的最大深度
'num_leaves':500,#叶子节点数
'max_bins':255,#分箱数s
'lambda_l1':1e-4,#l1正则化权重
'lambda_l2':1e-4,#l2正则化权重
'min_data_in_leaf':5,
'bagging_freq':5,
'bagging_fraction':0.5
}

trn_x = trn_df.drop(['policy_no','y'], axis=1)
trn_y = trn_df['y']

val_x = dev_df.drop(['policy_no','y'], axis=1)
val_y = dev_df['y']

trn_data = lgb.Dataset(trn_x, trn_y,categorical_feature=object_cols)
val_data = lgb.Dataset(val_x, val_y,categorical_feature=object_cols)

reg = lgb.train(params = params,
                train_set = trn_data,
                num_boost_round = 500,#最大树数量
                early_stopping_rounds = 100,#如何验证集效果在20轮中没有明显变好，就终止
                feval=eval_function,
                valid_sets = [val_data])

In [7]:
test_x = test_df.drop(['policy_no','y'], axis=1)
test_y = test_df['y']
print("训练集预测结果",eval(trn_y.values,reg.predict(trn_x)))
print("验证集预测结果",eval(val_y.values,reg.predict(val_x)))
print("测试集预测结果",eval(test_y.values,reg.predict(test_x)))

训练集预测结果 0.72
验证集预测结果 0.5
测试集预测结果 0.5


这时,    
偏差=1.0-0.72=0.28  
方差=0.72-0.5=0.22  
方差下降了大概15%，偏差没变，这里需要注意一个问题，并不是无脑添加数据，模型效果就能得到提升，模型效果与训练样本容量通常有如下规律     
![avatar](./pic/2.jpg)  